In [1]:
import pandas as pd
import numpy as np
import missingno as mn

import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob

import glob
import os
from sklearn.model_selection import cross_val_score

import abc
import numpy as np

import pandas as pd
import abc
import re
import nltk
from sklearn import model_selection 
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [18]:

import time
import numpy as np

import datetime
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA

import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [20]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
import datetime
import missingno as mn
import pylab as plt
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.feature_selection import VarianceThreshold
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [5]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [4]:
import pandas as pd
import numpy as np
import missingno as mn

In [5]:
featureDF_nw = pd.read_csv('nw_feature_woText_MSFT.csv')
featureDF_tw = pd.read_csv('tw_feature_woText_MSFT.csv')
targetDF_ = pd.read_csv('FIwithTargetwithFTT.csv')

In [6]:
def createComparableDataset(_df):
    _df.date = pd.to_datetime(_df.date)
    _df = _df[_df['date'] >= pd.to_datetime('2015/05/04')]
    _df = _df[_df['date'] <= pd.to_datetime('2019/03/28')]
    return _df

In [7]:
def createTrainingAndTestingSet(_df):
    train = _df[_df['date'] < pd.to_datetime('2019/01/01')]
    test = _df[_df['date'] >= pd.to_datetime('2019/01/01')]
    return train ,test

In [8]:
def getStartEndDateShape(dfz):
    startDate = str(min(dfz['date']))
    endDate = str(max(dfz['date']))
    print('startDate', startDate)
    print('endDate', endDate)
    print('shape', dfz.shape)
    print('---------------------------')

In [9]:
def getFeaturesAndTargetRemoveDate(_df):
    try:
        train_y = _df['target']
        train = _df.drop(['date','target'],axis =1)
    except:
        train_y = _df['Target']
        train = _df.drop(['date','Target'],axis =1)        
    return train, train_y

In [12]:
getStartEndDateShape(featureDF_nw)
getStartEndDateShape(featureDF_tw)
getStartEndDateShape(targetDF_)

startDate 2004/07/21
endDate 2019/03/28
shape (4020, 24)
---------------------------
startDate 2015/05/04
endDate 2019/04/04
shape (1426, 25)
---------------------------
startDate 2003/04/30
endDate 2019/04/03
shape (4010, 32)
---------------------------


In [13]:
targtWithDateForJoin =pd.DataFrame()
targtWithDateForJoin['date'] = targetDF_.date.tolist()
targtWithDateForJoin['target'] = targetDF_.Target.tolist()

# Formulation of overall data

In [14]:
featureDF_nw.date = pd.to_datetime(featureDF_nw.date)
targtWithDateForJoin.date = pd.to_datetime(targtWithDateForJoin.date)
featureDF_tw.date = pd.to_datetime(featureDF_tw.date)
targetDF_.date = pd.to_datetime(targetDF_.date)

In [15]:
featureDF_nw= createComparableDataset(featureDF_nw)
targtWithDateForJoin=createComparableDataset(targtWithDateForJoin)
featureDF_tw=createComparableDataset(featureDF_tw)
targetDF_=createComparableDataset(targetDF_)
targetDF_.rename(columns={'Target':'target'}, inplace=True)

In [16]:
getStartEndDateShape(featureDF_nw)
getStartEndDateShape(featureDF_tw)
getStartEndDateShape(targetDF_)

startDate 2015-05-04 00:00:00
endDate 2019-03-28 00:00:00
shape (1027, 24)
---------------------------
startDate 2015-05-04 00:00:00
endDate 2019-03-28 00:00:00
shape (1419, 25)
---------------------------
startDate 2015-05-04 00:00:00
endDate 2019-03-28 00:00:00
shape (983, 32)
---------------------------


In [17]:
result =pd.merge( targetDF_,featureDF_nw, how='outer', on='date')

In [18]:
getStartEndDateShape(result)

startDate 2015-05-04 00:00:00
endDate 2019-03-28 00:00:00
shape (1171, 55)
---------------------------


In [19]:
result =pd.merge( result,featureDF_tw, how='outer', on='date')

In [20]:
getStartEndDateShape(result)

startDate 2015-05-04 00:00:00
endDate 2019-03-28 00:00:00
shape (1423, 79)
---------------------------


In [21]:
result.sort_values(by='date',inplace=True) 

In [22]:
result = result.ffill()
result.fillna(0,inplace=True)

In [23]:
result.head(5)

,Open,High,Low,Close,Adj Close,Volume,sma2,sma2_1,sma2_increment,sma2_1_increment,...,disgust_y,anticipation_y,fear_y,trust_y,tb_polarity_y,tb_subjectivity_y,hiv4_positive_y,hiv4_negative_y,hiv4_polarity_y,hiv4_subjectivity_y
0,48.369999,48.869999,48.180000,48.240002,44.072769,34039500.0,48.475001,48.640002,-0.165001,-0.069999,...,0.0,0.002845,0.0,0.000000,2.400000,2.383333,5.0,2.0,2.333331,0.589387
1,47.820000,48.160000,47.310001,47.599998,43.488049,50369200.0,48.095000,48.475001,-0.380001,-0.165001,...,0.0,0.003759,0.0,0.000000,0.000000,0.300000,3.0,2.0,0.666666,0.326007
2,47.570000,47.770000,46.020000,46.279999,42.282074,52433000.0,47.695000,48.095000,-0.400000,-0.380001,...,0.0,0.000522,0.0,0.000000,1.250947,7.171212,19.0,5.0,7.999995,3.123412
3,46.270000,47.090000,46.160000,46.700001,42.665791,32971700.0,46.920000,47.695000,-0.775000,-0.400000,...,0.0,0.000573,0.0,0.000573,0.322159,3.534091,2.0,7.0,-3.999997,1.258474
4,47.549999,47.980000,47.520000,47.750000,43.625092,35364900.0,46.909999,46.920000,-0.010001,-0.775000,...,0.0,0.000548,0.0,0.000000,0.325000,0.962500,11.0,6.0,2.333332,1.455759


In [24]:
result.tail(5)

,Open,High,Low,Close,Adj Close,Volume,sma2,sma2_1,sma2_increment,sma2_1_increment,...,disgust_y,anticipation_y,fear_y,trust_y,tb_polarity_y,tb_subjectivity_y,hiv4_positive_y,hiv4_negative_y,hiv4_polarity_y,hiv4_subjectivity_y
1092,119.500000,119.589996,117.040001,117.050003,117.050003,33624500.0,118.319999,117.264999,1.055001,-0.474998,...,12.0,0.001662,0.000345,0.001552,54.011847,133.276238,565.0,273.0,99.701533,75.265758
979,116.559998,118.010002,116.320000,117.660004,117.660004,27067100.0,118.029999,118.319999,-0.290000,1.055001,...,4.0,0.001268,0.000215,0.000933,21.654621,77.138890,361.0,156.0,62.624211,41.045214
980,118.620003,118.709999,116.849998,117.910004,117.910004,26097700.0,117.590000,118.029999,-0.439999,-0.290000,...,19.0,0.001762,0.000370,0.001193,98.739210,286.580670,1363.0,625.0,248.350599,147.048014
981,117.879997,118.209999,115.519997,116.769997,116.769997,22733400.0,118.250000,117.590000,0.659999,-0.439999,...,17.0,0.001513,0.000332,0.001175,110.125758,283.081651,1428.0,565.0,244.546184,132.279292
982,117.440002,117.580002,116.129997,116.930000,116.930000,18334800.0,117.659999,118.250000,-0.590001,0.659999,...,27.0,0.001972,0.000315,0.001134,101.462943,299.296855,1406.0,590.0,267.852539,148.186288


In [25]:
# Save data with without scaling
result.to_csv('OverAllDataset_wo_scaling.csv',index= False)

In [26]:
getStartEndDateShape(result)

startDate 2015-05-04 00:00:00
endDate 2019-03-28 00:00:00
shape (1423, 79)
---------------------------


In [27]:
date= result.date.tolist()
target = result.target.tolist()
result.drop(['date','target'],axis =1,inplace=True)

In [28]:
scale = StandardScaler()
result = pd.DataFrame(scale.fit_transform(result.values), columns=result.columns, index=result.index)

In [29]:
result['date'] = date
result['Target'] = target

In [30]:
# Save data with without scaling                   
result.to_csv('OverAllDataset_with_scaling.csv',index= False)


In [31]:
getStartEndDateShape(result)

startDate 2015-05-04 00:00:00
endDate 2019-03-28 00:00:00
shape (1423, 79)
---------------------------


# Training and Test Split

In [32]:
train,test = createTrainingAndTestingSet(result)

In [33]:
getStartEndDateShape(train)
getStartEndDateShape(test)

startDate 2015-05-04 00:00:00
endDate 2018-12-31 00:00:00
shape (1336, 79)
---------------------------
startDate 2019-01-01 00:00:00
endDate 2019-03-28 00:00:00
shape (87, 79)
---------------------------


In [34]:
train.to_csv('train_tw_nw_fi.csv',index=False)
test.to_csv('test_tw_nw_fi.csv',index=False)

In [35]:
nonScaledDataset = pd.read_csv('OverAllDataset_wo_scaling.csv')
nonScaledDataset.date = pd.to_datetime(nonScaledDataset.date)

In [36]:
train_en, test_en = createTrainingAndTestingSet(nonScaledDataset)

In [37]:
getStartEndDateShape(train_en)
getStartEndDateShape(test_en)

startDate 2015-05-04 00:00:00
endDate 2018-12-31 00:00:00
shape (1336, 79)
---------------------------
startDate 2019-01-01 00:00:00
endDate 2019-03-28 00:00:00
shape (87, 79)
---------------------------


In [38]:
train_en.to_csv('train_ns_tw_nw_fi.csv',index= False)
test_en.to_csv('test_ns_tw_nw_fi.csv',index= False)

# NEWS RandomForest- NonScaled

In [39]:
featureDF_nw.date = pd.to_datetime(featureDF_nw.date)
targtWithDateForJoin.date = pd.to_datetime(targtWithDateForJoin.date)
result =pd.merge(featureDF_nw, targtWithDateForJoin, how='left', on='date')

In [40]:
result.sort_values(by='date',inplace=True) 

In [41]:
result['target'] = result['target'].ffill()

In [42]:
#mn.matrix(result)

In [43]:
#train and test split
train_nw, test_nw = createTrainingAndTestingSet(result)

train_nw_y = train_nw['target']
test_nw_y = test_nw['target']


In [44]:
getStartEndDateShape(train_nw)
getStartEndDateShape(test_nw)

startDate 2015-05-04 00:00:00
endDate 2018-12-30 00:00:00
shape (961, 25)
---------------------------
startDate 2019-01-03 00:00:00
endDate 2019-03-28 00:00:00
shape (66, 25)
---------------------------


In [45]:

train_nw.drop(['date','target'],axis=1,inplace=True)
test_nw.drop(['date','target'],axis=1,inplace=True)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [46]:
print('train_nw',train_nw.shape,'test_nw',test_nw.shape,'len(train_nw_y)',len(train_nw_y),'len(test_nw_y)',len(test_nw_y))

train_nw (961, 23) test_nw (66, 23) len(train_nw_y) 961 len(test_nw_y) 66


In [47]:
def NaivePredition(y_test):
    
    y_testCheck = y_test.copy()
    y_pred = list()
    
    for counter in range(len(y_testCheck)-1):
        y_pred.append(y_testCheck[counter])
    
    y_testCheck = y_testCheck[1:]
    print ('Model accuracy = %.3f' % accuracy_score(y_pred,y_testCheck))

    print('classification_score\n',classification_report(y_pred, y_testCheck))

In [48]:
NaivePredition(list(test_nw_y))

Model accuracy = 0.692
classification_score
               precision    recall  f1-score   support

         0.0       0.66      0.66      0.66        29
         1.0       0.72      0.72      0.72        36

   micro avg       0.69      0.69      0.69        65
   macro avg       0.69      0.69      0.69        65
weighted avg       0.69      0.69      0.69        65



In [16]:
# Naive Bayes for Baseline
from sklearn.naive_bayes import GaussianNB
def NaiveBayes(train_features , train_labels, test_features,test_labels):
    gnb = GaussianNB()
    y_pred = gnb.fit(train_features, train_labels).predict(test_features)
    print('accuracy_score',accuracy_score(y_pred, test_labels))
    print('classification_score\n',classification_report(y_pred, test_labels))
    

In [50]:
def plotImportanceOfFeature(features,clf):
    importances = clf.feature_importances_
    indices = np.argsort(importances)

    plt.title('Feature Importances')
    plt.barh(range(len(indices)), importances[indices], color='b', align='center')
    plt.yticks(range(len(indices)), [features[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.show()

In [51]:
def corelationPlot (df_):
    corr = df_.corr()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
    fig.colorbar(cax)
    ticks = np.arange(0,len(df_.columns),1)
    ax.set_xticks(ticks)
    plt.xticks(rotation=90)
    ax.set_yticks(ticks)
    ax.set_xticklabels(df_.columns)
    ax.set_yticklabels(df_.columns)
    plt.show()

In [52]:
def vif_RemoveHiglyCorelatedFeatures(_df,_dfTest,corPlot = False):
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(_df.values, i) for i in range(_df.values.shape[1])]
    vif.round(1)
    while max(vif["VIF Factor"])>10:
        vif = pd.DataFrame()
        vif["VIF Factor"] = [variance_inflation_factor(_df.values, i) for i in range(_df.values.shape[1])]
        vif.round(1)
        dropCol = [_df.columns[np.argmax(vif["VIF Factor"])]]
        _df.drop(columns=dropCol, inplace=True)
        _dfTest.drop(columns=dropCol, inplace=True)
        if corPlot:
            corelationPlot (_df)

In [53]:
vif_RemoveHiglyCorelatedFeatures(train_nw,test_nw)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


In [54]:
print('train_nw',train_nw.shape,'test_nw',test_nw.shape,'len(train_nw_y)',len(train_nw_y),'len(test_nw_y)',len(test_nw_y))

train_nw (961, 16) test_nw (66, 16) len(train_nw_y) 961 len(test_nw_y) 66


In [55]:
def cCMatrixAccRF(train_features , train_labels, test_features,test_labels, features, dispImpOfFeature = False):
#         train_features, test_features, train_labels, test_labels= model_selection.train_test_split( featureDF , targetValue, test_size=0.1, random_state=0)

        rf_model =RandomForestClassifier(n_estimators=5000, max_depth=4,random_state=0)
        rf_model.fit(train_features, train_labels)
        y_predicted=rf_model.predict(test_features)
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++' )
        
        if dispImpOfFeature:
            plotImportanceOfFeature(features, rf_model)
        print('confusion_matrix',confusion_matrix(y_true=test_labels, y_pred=y_predicted))
        print('number of datapoints in Up Class', test_labels.value_counts()[1])
        print('number of datapoints in Down Class', test_labels.value_counts()[0])
        print('accuracy_score',accuracy_score(y_predicted, test_labels))
        print('classification_score\n',classification_report(y_predicted, test_labels))
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
#         return rf_model.predict(train_features),rf_model.predict(test_features)

In [56]:
def cCMatrixAccAdaboost(train_features , train_labels, test_features,test_labels):
#         train_features, test_features, train_labels, test_labels= model_selection.train_test_split( featureDF , targetValue, test_size=0.1, random_state=0)

        abc_model =AdaBoostClassifier(n_estimators=5000, random_state=0)
        abc_model.fit(train_features, train_labels)
        y_predicted=abc_model.predict(test_features)
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++' )
        
        print('confusion_matrix',confusion_matrix(y_true=test_labels, y_pred=y_predicted))
        print('number of datapoints in Up Class', test_labels.value_counts()[1])
        print('number of datapoints in Down Class', test_labels.value_counts()[0])
        print('accuracy_score',accuracy_score(y_predicted, test_labels))
        print('classification_score\n',classification_report(y_predicted, test_labels))
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [57]:
def cCMatrixAccGradientTreeBosting(train_features , train_labels, test_features,test_labels):
#         train_features, test_features, train_labels, test_labels= model_selection.train_test_split( featureDF , targetValue, test_size=0.1, random_state=0)

        gbc_model =GradientBoostingClassifier(n_estimators=5000, learning_rate=1.0, max_depth=4,random_state=0)
        gbc_model.fit(train_features, train_labels)
        y_predicted=gbc_model.predict(test_features)
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++' )
        
        print('confusion_matrix',confusion_matrix(y_true=test_labels, y_pred=y_predicted))
        print('number of datapoints in Up Class', test_labels.value_counts()[1])
        print('number of datapoints in Down Class', test_labels.value_counts()[0])
        print('accuracy_score',accuracy_score(y_predicted, test_labels))
        print('classification_score\n',classification_report(y_predicted, test_labels))
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [58]:
def cCMatrixAccXGB(X_train , y_train, X_test,y_test):
    
    regressor = xgb.XGBRegressor(gamma=0.0,n_estimators=150,base_score=0.7,colsample_bytree=1,learning_rate=0.01)

    xgbModel = regressor.fit(X_train,y_train)

    y_predicted = xgbModel.predict(X_test)
    y_predicted_binary = [1 if yp >=0.5 else 0 for yp in y_predicted] # (y_predicted > 0.5)

    print ('Model accuracy = %.3f' % accuracy_score(y_test,y_predicted_binary))

    print('classification_score\n',classification_report(y_test, y_predicted_binary))
#     return y_predicted_binary

In [59]:
def cCMatrixAccVotingClasifier(train_features , train_labels, test_features,test_labels):

        rf_model =RandomForestClassifier(n_estimators=5000, max_depth=4,random_state=0)
        abc_model =AdaBoostClassifier(n_estimators=5000, random_state=0)
        gbc_model =GradientBoostingClassifier(n_estimators=5000, learning_rate=1.0, max_depth=4,random_state=0)
        lr_model = LogisticRegression(solver='lbfgs', multi_class='ovr',random_state=1)
        gnb_model = GaussianNB()
        
        eclf = VotingClassifier(estimators=[('lr', lr_model), ('rf', rf_model), ('gnb', gnb_model), ('abc', abc_model), ('gbc', gbc_model)], voting='soft')
        
        for clf, label in zip([lr_model, rf_model, gnb_model,abc_model,gbc_model, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'AdaBoostClassifier','GradientBoostingClassifier','Ensemble']):
            scores = cross_val_score(clf, train_features, train_labels, cv=5, scoring='accuracy')
            print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [60]:
print('train_nw',train_nw.shape,'test_nw',test_nw.shape,'len(train_nw_y)',len(train_nw_y),'len(test_nw_y)',len(test_nw_y))

train_nw (961, 16) test_nw (66, 16) len(train_nw_y) 961 len(test_nw_y) 66


In [61]:
NaiveBayes(train_nw , train_nw_y, test_nw,test_nw_y)

accuracy_score 0.6212121212121212
classification_score
               precision    recall  f1-score   support

         0.0       0.28      0.67      0.39        12
         1.0       0.89      0.61      0.73        54

   micro avg       0.62      0.62      0.62        66
   macro avg       0.58      0.64      0.56        66
weighted avg       0.78      0.62      0.66        66



In [62]:
features = list(train_nw.columns)
cCMatrixAccRF(train_nw , train_nw_y, test_nw,test_nw_y, features)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[ 1 28]
 [ 0 37]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5757575757575758
classification_score
               precision    recall  f1-score   support

         0.0       0.03      1.00      0.07         1
         1.0       1.00      0.57      0.73        65

   micro avg       0.58      0.58      0.58        66
   macro avg       0.52      0.78      0.40        66
weighted avg       0.99      0.58      0.72        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [63]:
cCMatrixAccAdaboost(train_nw , train_nw_y, test_nw,test_nw_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[11 18]
 [16 21]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.48484848484848486
classification_score
               precision    recall  f1-score   support

         0.0       0.38      0.41      0.39        27
         1.0       0.57      0.54      0.55        39

   micro avg       0.48      0.48      0.48        66
   macro avg       0.47      0.47      0.47        66
weighted avg       0.49      0.48      0.49        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [64]:
cCMatrixAccGradientTreeBosting(train_nw , train_nw_y, test_nw,test_nw_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[13 16]
 [16 21]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5151515151515151
classification_score
               precision    recall  f1-score   support

         0.0       0.45      0.45      0.45        29
         1.0       0.57      0.57      0.57        37

   micro avg       0.52      0.52      0.52        66
   macro avg       0.51      0.51      0.51        66
weighted avg       0.52      0.52      0.52        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [65]:
cCMatrixAccVotingClasifier(train_nw , train_nw_y, test_nw,test_nw_y)

Accuracy: 0.53 (+/- 0.01) [Logistic Regression]
Accuracy: 0.53 (+/- 0.01) [Random Forest]
Accuracy: 0.49 (+/- 0.05) [naive Bayes]
Accuracy: 0.52 (+/- 0.03) [AdaBoostClassifier]
Accuracy: 0.52 (+/- 0.03) [GradientBoostingClassifier]
Accuracy: 0.51 (+/- 0.03) [Ensemble]


In [66]:
cCMatrixAccXGB(train_nw , train_nw_y, test_nw,test_nw_y)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model accuracy = 0.591
classification_score
               precision    recall  f1-score   support

         0.0       1.00      0.07      0.13        29
         1.0       0.58      1.00      0.73        37

   micro avg       0.59      0.59      0.59        66
   macro avg       0.79      0.53      0.43        66
weighted avg       0.76      0.59      0.47        66



# NEWS RandomForest- Scaled

In [67]:
print('train_nw',train_nw.shape,'test_nw',test_nw.shape,'len(train_nw_y)',len(train_nw_y),'len(test_nw_y)',len(test_nw_y))

train_nw (961, 16) test_nw (66, 16) len(train_nw_y) 961 len(test_nw_y) 66


In [68]:
scale = StandardScaler()
train_nw = pd.DataFrame(scale.fit_transform(train_nw.values), columns=train_nw.columns, index=train_nw.index)
scale = StandardScaler()
test_nw = pd.DataFrame(scale.fit_transform(test_nw.values), columns=test_nw.columns, index=test_nw.index)

In [69]:
vif_RemoveHiglyCorelatedFeatures(train_nw,test_nw)
print('train_nw',train_nw.shape,'test_nw',test_nw.shape,'len(train_nw_y)',len(train_nw_y),'len(test_nw_y)',len(test_nw_y))

train_nw (961, 16) test_nw (66, 16) len(train_nw_y) 961 len(test_nw_y) 66


In [70]:
NaiveBayes(train_nw , train_nw_y, test_nw,test_nw_y)

accuracy_score 0.5757575757575758
classification_score
               precision    recall  f1-score   support

         0.0       0.72      0.51      0.60        41
         1.0       0.46      0.68      0.55        25

   micro avg       0.58      0.58      0.58        66
   macro avg       0.59      0.60      0.57        66
weighted avg       0.62      0.58      0.58        66



In [71]:
features = list(train_nw.columns)
temp=cCMatrixAccRF(train_nw , train_nw_y, test_nw,test_nw_y, features)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[ 2 27]
 [ 2 35]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5606060606060606
classification_score
               precision    recall  f1-score   support

         0.0       0.07      0.50      0.12         4
         1.0       0.95      0.56      0.71        62

   micro avg       0.56      0.56      0.56        66
   macro avg       0.51      0.53      0.41        66
weighted avg       0.89      0.56      0.67        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [72]:
cCMatrixAccAdaboost(train_nw , train_nw_y, test_nw,test_nw_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[ 8 21]
 [11 26]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5151515151515151
classification_score
               precision    recall  f1-score   support

         0.0       0.28      0.42      0.33        19
         1.0       0.70      0.55      0.62        47

   micro avg       0.52      0.52      0.52        66
   macro avg       0.49      0.49      0.48        66
weighted avg       0.58      0.52      0.54        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [73]:
cCMatrixAccGradientTreeBosting(train_nw , train_nw_y, test_nw,test_nw_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[10 19]
 [13 24]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5151515151515151
classification_score
               precision    recall  f1-score   support

         0.0       0.34      0.43      0.38        23
         1.0       0.65      0.56      0.60        43

   micro avg       0.52      0.52      0.52        66
   macro avg       0.50      0.50      0.49        66
weighted avg       0.54      0.52      0.52        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [74]:
cCMatrixAccVotingClasifier(train_nw , train_nw_y, test_nw,test_nw_y)

Accuracy: 0.52 (+/- 0.02) [Logistic Regression]
Accuracy: 0.53 (+/- 0.01) [Random Forest]
Accuracy: 0.48 (+/- 0.04) [naive Bayes]
Accuracy: 0.51 (+/- 0.02) [AdaBoostClassifier]
Accuracy: 0.51 (+/- 0.01) [GradientBoostingClassifier]
Accuracy: 0.50 (+/- 0.01) [Ensemble]


In [75]:
cCMatrixAccXGB(train_nw , train_nw_y, test_nw,test_nw_y)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model accuracy = 0.561
classification_score
               precision    recall  f1-score   support

         0.0       0.50      0.07      0.12        29
         1.0       0.56      0.95      0.71        37

   micro avg       0.56      0.56      0.56        66
   macro avg       0.53      0.51      0.41        66
weighted avg       0.54      0.56      0.45        66



# Twitter RandomForest- NonScaled

In [76]:
featureDF_tw.date = pd.to_datetime(featureDF_tw.date)
targtWithDateForJoin.date = pd.to_datetime(targtWithDateForJoin.date)
result =pd.merge(featureDF_tw, targtWithDateForJoin, how='left', on='date')

result.sort_values(by='date',inplace=True) 

result['target'] = result['target'].ffill()

In [77]:
train_tw, test_tw = createTrainingAndTestingSet(result)

In [78]:
train_tw_y = train_tw['target']
test_tw_y = test_tw['target']

In [79]:
getStartEndDateShape(train_tw)
getStartEndDateShape(test_tw)

startDate 2015-05-04 00:00:00
endDate 2018-12-31 00:00:00
shape (1332, 26)
---------------------------
startDate 2019-01-01 00:00:00
endDate 2019-03-28 00:00:00
shape (87, 26)
---------------------------


In [80]:
train_tw.drop(['date','target'],axis=1,inplace=True)
test_tw.drop(['date','target'],axis=1,inplace=True)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [81]:
NaivePredition(list(test_tw_y))

Model accuracy = 0.756
classification_score
               precision    recall  f1-score   support

         0.0       0.71      0.69      0.70        36
         1.0       0.78      0.80      0.79        50

   micro avg       0.76      0.76      0.76        86
   macro avg       0.75      0.75      0.75        86
weighted avg       0.75      0.76      0.76        86



In [82]:
NaiveBayes(train_tw , train_tw_y, test_tw,test_tw_y)

accuracy_score 0.5862068965517241
classification_score
               precision    recall  f1-score   support

         0.0       0.06      0.50      0.10         4
         1.0       0.96      0.59      0.73        83

   micro avg       0.59      0.59      0.59        87
   macro avg       0.51      0.55      0.42        87
weighted avg       0.92      0.59      0.70        87



In [83]:
print('train_nw',train_nw.shape,'test_nw',test_nw.shape,'len(train_nw_y)',len(train_nw_y),'len(test_nw_y)',len(test_nw_y))

train_nw (961, 16) test_nw (66, 16) len(train_nw_y) 961 len(test_nw_y) 66


In [84]:
vif_RemoveHiglyCorelatedFeatures(train_nw,test_nw)

In [85]:
print('train_nw',train_nw.shape,'test_nw',test_nw.shape,'len(train_nw_y)',len(train_nw_y),'len(test_nw_y)',len(test_nw_y))

train_nw (961, 16) test_nw (66, 16) len(train_nw_y) 961 len(test_nw_y) 66


In [86]:
features = list(train_nw.columns)
cCMatrixAccRF(train_nw , train_nw_y, test_nw,test_nw_y, features)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[ 2 27]
 [ 2 35]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5606060606060606
classification_score
               precision    recall  f1-score   support

         0.0       0.07      0.50      0.12         4
         1.0       0.95      0.56      0.71        62

   micro avg       0.56      0.56      0.56        66
   macro avg       0.51      0.53      0.41        66
weighted avg       0.89      0.56      0.67        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [87]:
cCMatrixAccAdaboost(train_nw , train_nw_y, test_nw,test_nw_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[ 8 21]
 [11 26]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5151515151515151
classification_score
               precision    recall  f1-score   support

         0.0       0.28      0.42      0.33        19
         1.0       0.70      0.55      0.62        47

   micro avg       0.52      0.52      0.52        66
   macro avg       0.49      0.49      0.48        66
weighted avg       0.58      0.52      0.54        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [88]:
cCMatrixAccGradientTreeBosting(train_nw , train_nw_y, test_nw,test_nw_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[10 19]
 [13 24]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5151515151515151
classification_score
               precision    recall  f1-score   support

         0.0       0.34      0.43      0.38        23
         1.0       0.65      0.56      0.60        43

   micro avg       0.52      0.52      0.52        66
   macro avg       0.50      0.50      0.49        66
weighted avg       0.54      0.52      0.52        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [89]:
cCMatrixAccVotingClasifier(train_nw , train_nw_y, test_nw,test_nw_y)

Accuracy: 0.52 (+/- 0.02) [Logistic Regression]
Accuracy: 0.53 (+/- 0.01) [Random Forest]
Accuracy: 0.48 (+/- 0.04) [naive Bayes]
Accuracy: 0.51 (+/- 0.02) [AdaBoostClassifier]
Accuracy: 0.51 (+/- 0.01) [GradientBoostingClassifier]
Accuracy: 0.50 (+/- 0.01) [Ensemble]


In [90]:
cCMatrixAccXGB(train_nw , train_nw_y, test_nw,test_nw_y)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model accuracy = 0.561
classification_score
               precision    recall  f1-score   support

         0.0       0.50      0.07      0.12        29
         1.0       0.56      0.95      0.71        37

   micro avg       0.56      0.56      0.56        66
   macro avg       0.53      0.51      0.41        66
weighted avg       0.54      0.56      0.45        66



# Twitter RandomForest- Scaled

In [91]:
scale = StandardScaler()
train_tw = pd.DataFrame(scale.fit_transform(train_tw.values), columns=train_tw.columns, index=train_tw.index)
scale = StandardScaler()
test_tw = pd.DataFrame(scale.fit_transform(test_tw.values), columns=test_tw.columns, index=test_tw.index)

In [92]:
print('train_tw',train_tw.shape,'test_tw',test_tw.shape,'len(train_tw_y)',len(train_tw_y),'len(test_tw_y)',len(test_tw_y))

train_tw (1332, 24) test_tw (87, 24) len(train_tw_y) 1332 len(test_tw_y) 87


In [93]:
vif_RemoveHiglyCorelatedFeatures(train_tw,test_tw)
print('train_tw',train_tw.shape,'test_tw',test_tw.shape,'len(train_tw_y)',len(train_tw_y),'len(test_tw_y)',len(test_tw_y))

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


train_tw (1332, 12) test_tw (87, 12) len(train_tw_y) 1332 len(test_tw_y) 87


In [94]:
NaiveBayes(train_tw , train_tw_y, test_tw,test_tw_y)

accuracy_score 0.4482758620689655
classification_score
               precision    recall  f1-score   support

         0.0       0.36      0.34      0.35        38
         1.0       0.51      0.53      0.52        49

   micro avg       0.45      0.45      0.45        87
   macro avg       0.44      0.44      0.44        87
weighted avg       0.44      0.45      0.45        87



In [95]:
features = list(train_tw.columns)
temp = cCMatrixAccRF(train_tw , train_tw_y, test_tw,test_tw_y, features)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[ 9 27]
 [12 39]]
number of datapoints in Up Class 51
number of datapoints in Down Class 36
accuracy_score 0.5517241379310345
classification_score
               precision    recall  f1-score   support

         0.0       0.25      0.43      0.32        21
         1.0       0.76      0.59      0.67        66

   micro avg       0.55      0.55      0.55        87
   macro avg       0.51      0.51      0.49        87
weighted avg       0.64      0.55      0.58        87

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [96]:
cCMatrixAccAdaboost(train_tw , train_tw_y, test_tw,test_tw_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[10 26]
 [21 30]]
number of datapoints in Up Class 51
number of datapoints in Down Class 36
accuracy_score 0.45977011494252873
classification_score
               precision    recall  f1-score   support

         0.0       0.28      0.32      0.30        31
         1.0       0.59      0.54      0.56        56

   micro avg       0.46      0.46      0.46        87
   macro avg       0.43      0.43      0.43        87
weighted avg       0.48      0.46      0.47        87

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [97]:
cCMatrixAccGradientTreeBosting(train_tw , train_tw_y, test_tw,test_tw_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[11 25]
 [18 33]]
number of datapoints in Up Class 51
number of datapoints in Down Class 36
accuracy_score 0.5057471264367817
classification_score
               precision    recall  f1-score   support

         0.0       0.31      0.38      0.34        29
         1.0       0.65      0.57      0.61        58

   micro avg       0.51      0.51      0.51        87
   macro avg       0.48      0.47      0.47        87
weighted avg       0.53      0.51      0.52        87

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [98]:
cCMatrixAccVotingClasifier(train_tw , train_tw_y, test_tw,test_tw_y)

Accuracy: 0.54 (+/- 0.05) [Logistic Regression]
Accuracy: 0.52 (+/- 0.04) [Random Forest]
Accuracy: 0.53 (+/- 0.04) [naive Bayes]
Accuracy: 0.51 (+/- 0.04) [AdaBoostClassifier]
Accuracy: 0.52 (+/- 0.04) [GradientBoostingClassifier]
Accuracy: 0.53 (+/- 0.03) [Ensemble]


In [99]:
cCMatrixAccXGB(train_tw , train_tw_y, test_tw,test_tw_y)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model accuracy = 0.563
classification_score
               precision    recall  f1-score   support

         0.0       0.44      0.22      0.30        36
         1.0       0.59      0.80      0.68        51

   micro avg       0.56      0.56      0.56        87
   macro avg       0.52      0.51      0.49        87
weighted avg       0.53      0.56      0.52        87



# Financial Indicator RandomForest- NonScaled

In [100]:
fi_df_copy = targetDF_.copy()

In [101]:
train_fi, test_fi= createTrainingAndTestingSet(fi_df_copy)

train_fi_y = train_fi['target']
test_fi_y = test_fi['target']

getStartEndDateShape(train_fi)
getStartEndDateShape(test_fi)

train_fi.drop(['date','target'],axis=1,inplace=True)
test_fi.drop(['date','target'],axis=1,inplace=True)

startDate 2015-05-04 00:00:00
endDate 2018-12-31 00:00:00
shape (923, 32)
---------------------------
startDate 2019-01-02 00:00:00
endDate 2019-03-28 00:00:00
shape (60, 32)
---------------------------


C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [102]:
print('train_fi',train_fi.shape,'test_fi',test_fi.shape,'len(train_fi_y)',len(train_fi_y),'len(test_fi_y)',len(test_fi_y))

train_fi (923, 30) test_fi (60, 30) len(train_fi_y) 923 len(test_fi_y) 60


In [103]:
vif_RemoveHiglyCorelatedFeatures(train_fi,test_fi)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\statsmodels\stats\outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


In [104]:
NaivePredition(list(test_fi_y))

Model accuracy = 0.644
classification_score
               precision    recall  f1-score   support

           0       0.58      0.56      0.57        25
           1       0.69      0.71      0.70        34

   micro avg       0.64      0.64      0.64        59
   macro avg       0.63      0.63      0.63        59
weighted avg       0.64      0.64      0.64        59



In [105]:
NaiveBayes(train_fi , train_fi_y, test_fi,test_fi_y)

accuracy_score 0.5
classification_score
               precision    recall  f1-score   support

           0       0.40      0.40      0.40        25
           1       0.57      0.57      0.57        35

   micro avg       0.50      0.50      0.50        60
   macro avg       0.49      0.49      0.49        60
weighted avg       0.50      0.50      0.50        60



In [106]:
print('train_fi',train_fi.shape,'test_fi',test_fi.shape,'len(train_fi_y)',len(train_fi_y),'len(test_fi_y)',len(test_fi_y))

train_fi (923, 9) test_fi (60, 9) len(train_fi_y) 923 len(test_fi_y) 60


In [107]:
features = list(train_nw.columns)
cCMatrixAccRF(train_nw , train_nw_y, test_nw,test_nw_y, features)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[ 2 27]
 [ 2 35]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5606060606060606
classification_score
               precision    recall  f1-score   support

         0.0       0.07      0.50      0.12         4
         1.0       0.95      0.56      0.71        62

   micro avg       0.56      0.56      0.56        66
   macro avg       0.51      0.53      0.41        66
weighted avg       0.89      0.56      0.67        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [108]:
cCMatrixAccAdaboost(train_nw , train_nw_y, test_nw,test_nw_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[ 8 21]
 [11 26]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5151515151515151
classification_score
               precision    recall  f1-score   support

         0.0       0.28      0.42      0.33        19
         1.0       0.70      0.55      0.62        47

   micro avg       0.52      0.52      0.52        66
   macro avg       0.49      0.49      0.48        66
weighted avg       0.58      0.52      0.54        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [109]:
cCMatrixAccGradientTreeBosting(train_nw , train_nw_y, test_nw,test_nw_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[10 19]
 [13 24]]
number of datapoints in Up Class 37
number of datapoints in Down Class 29
accuracy_score 0.5151515151515151
classification_score
               precision    recall  f1-score   support

         0.0       0.34      0.43      0.38        23
         1.0       0.65      0.56      0.60        43

   micro avg       0.52      0.52      0.52        66
   macro avg       0.50      0.50      0.49        66
weighted avg       0.54      0.52      0.52        66

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [110]:
cCMatrixAccVotingClasifier(train_nw , train_nw_y, test_nw,test_nw_y)

Accuracy: 0.52 (+/- 0.02) [Logistic Regression]
Accuracy: 0.53 (+/- 0.01) [Random Forest]
Accuracy: 0.48 (+/- 0.04) [naive Bayes]
Accuracy: 0.51 (+/- 0.02) [AdaBoostClassifier]
Accuracy: 0.51 (+/- 0.01) [GradientBoostingClassifier]
Accuracy: 0.50 (+/- 0.01) [Ensemble]


In [111]:
cCMatrixAccXGB(train_nw , train_nw_y, test_nw,test_nw_y)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model accuracy = 0.561
classification_score
               precision    recall  f1-score   support

         0.0       0.50      0.07      0.12        29
         1.0       0.56      0.95      0.71        37

   micro avg       0.56      0.56      0.56        66
   macro avg       0.53      0.51      0.41        66
weighted avg       0.54      0.56      0.45        66



# Financial Indicator RandomForest- Scaled

In [112]:
scale = StandardScaler()
train_fi = pd.DataFrame(scale.fit_transform(train_fi.values), columns=train_fi.columns, index=train_fi.index)
scale = StandardScaler()
test_fi = pd.DataFrame(scale.fit_transform(test_fi.values), columns=test_fi.columns, index=test_fi.index)

In [113]:
print('train_fi',train_fi.shape,'test_fi',test_fi.shape,'len(train_fi_y)',len(train_fi_y),'len(test_fi_y)',len(test_fi_y))

train_fi (923, 9) test_fi (60, 9) len(train_fi_y) 923 len(test_fi_y) 60


In [114]:
vif_RemoveHiglyCorelatedFeatures(train_fi,test_fi)
print('train_fi',train_fi.shape,'test_fi',test_fi.shape,'len(train_fi_y)',len(train_fi_y),'len(test_fi_y)',len(test_fi_y))

train_fi (923, 9) test_fi (60, 9) len(train_fi_y) 923 len(test_fi_y) 60


In [115]:
NaiveBayes(train_fi , train_fi_y, test_fi,test_fi_y)

accuracy_score 0.6333333333333333
classification_score
               precision    recall  f1-score   support

           0       0.60      0.56      0.58        27
           1       0.66      0.70      0.68        33

   micro avg       0.63      0.63      0.63        60
   macro avg       0.63      0.63      0.63        60
weighted avg       0.63      0.63      0.63        60



In [116]:
features = list(train_fi.columns)
temp = cCMatrixAccRF(train_fi , train_fi_y, test_fi,test_fi_y, features, False)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[22  3]
 [10 25]]
number of datapoints in Up Class 35
number of datapoints in Down Class 25
accuracy_score 0.7833333333333333
classification_score
               precision    recall  f1-score   support

           0       0.88      0.69      0.77        32
           1       0.71      0.89      0.79        28

   micro avg       0.78      0.78      0.78        60
   macro avg       0.80      0.79      0.78        60
weighted avg       0.80      0.78      0.78        60

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [117]:
cCMatrixAccAdaboost(train_fi , train_fi_y, test_fi,test_fi_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[15 10]
 [ 9 26]]
number of datapoints in Up Class 35
number of datapoints in Down Class 25
accuracy_score 0.6833333333333333
classification_score
               precision    recall  f1-score   support

           0       0.60      0.62      0.61        24
           1       0.74      0.72      0.73        36

   micro avg       0.68      0.68      0.68        60
   macro avg       0.67      0.67      0.67        60
weighted avg       0.69      0.68      0.68        60

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [118]:
cCMatrixAccGradientTreeBosting(train_fi , train_fi_y, test_fi,test_fi_y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[18  7]
 [ 8 27]]
number of datapoints in Up Class 35
number of datapoints in Down Class 25
accuracy_score 0.75
classification_score
               precision    recall  f1-score   support

           0       0.72      0.69      0.71        26
           1       0.77      0.79      0.78        34

   micro avg       0.75      0.75      0.75        60
   macro avg       0.75      0.74      0.74        60
weighted avg       0.75      0.75      0.75        60

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [119]:
cCMatrixAccVotingClasifier(train_fi , train_fi_y, test_fi,test_fi_y)

Accuracy: 0.75 (+/- 0.02) [Logistic Regression]
Accuracy: 0.75 (+/- 0.02) [Random Forest]
Accuracy: 0.63 (+/- 0.04) [naive Bayes]
Accuracy: 0.66 (+/- 0.01) [AdaBoostClassifier]
Accuracy: 0.70 (+/- 0.02) [GradientBoostingClassifier]
Accuracy: 0.73 (+/- 0.02) [Ensemble]


In [120]:
cCMatrixAccXGB(train_fi , train_fi_y, test_fi,test_fi_y)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model accuracy = 0.750
classification_score
               precision    recall  f1-score   support

           0       0.66      0.84      0.74        25
           1       0.86      0.69      0.76        35

   micro avg       0.75      0.75      0.75        60
   macro avg       0.76      0.76      0.75        60
weighted avg       0.77      0.75      0.75        60



# Ensemble Tw, News and FI -NonScaled

In [121]:
trainingData = pd.read_csv('train_ns_tw_nw_fi.csv')
testingData =  pd.read_csv('test_ns_tw_nw_fi.csv')

In [122]:
train , train_y = getFeaturesAndTargetRemoveDate(trainingData)
test,test_y = getFeaturesAndTargetRemoveDate(testingData)

In [123]:
train.head(2)

,Open,High,Low,Close,Adj Close,Volume,sma2,sma2_1,sma2_increment,sma2_1_increment,...,disgust_y,anticipation_y,fear_y,trust_y,tb_polarity_y,tb_subjectivity_y,hiv4_positive_y,hiv4_negative_y,hiv4_polarity_y,hiv4_subjectivity_y
0,48.369999,48.869999,48.180000,48.240002,44.072769,34039500.0,48.475001,48.640002,-0.165001,-0.069999,...,0.0,0.002845,0.0,0.0,2.4,2.383333,5.0,2.0,2.333331,0.589387
1,47.820000,48.160000,47.310001,47.599998,43.488049,50369200.0,48.095000,48.475001,-0.380001,-0.165001,...,0.0,0.003759,0.0,0.0,0.0,0.300000,3.0,2.0,0.666666,0.326007


In [124]:
NaivePredition(list(test_y))

Model accuracy = 0.756
classification_score
               precision    recall  f1-score   support

         0.0       0.71      0.69      0.70        36
         1.0       0.78      0.80      0.79        50

   micro avg       0.76      0.76      0.76        86
   macro avg       0.75      0.75      0.75        86
weighted avg       0.75      0.76      0.76        86



In [125]:
NaiveBayes(train , train_y, test,test_y)

accuracy_score 0.5862068965517241
classification_score
               precision    recall  f1-score   support

         0.0       0.22      0.50      0.31        16
         1.0       0.84      0.61      0.70        71

   micro avg       0.59      0.59      0.59        87
   macro avg       0.53      0.55      0.51        87
weighted avg       0.73      0.59      0.63        87



In [126]:
z = cCMatrixAccXGB(train , train_y, test,test_y)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model accuracy = 0.690
classification_score
               precision    recall  f1-score   support

         0.0       0.62      0.64      0.63        36
         1.0       0.74      0.73      0.73        51

   micro avg       0.69      0.69      0.69        87
   macro avg       0.68      0.68      0.68        87
weighted avg       0.69      0.69      0.69        87



In [127]:
features = list(train.columns)
k=cCMatrixAccRF(train , train_y, test,test_y, features,False)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[24 12]
 [12 39]]
number of datapoints in Up Class 51
number of datapoints in Down Class 36
accuracy_score 0.7241379310344828
classification_score
               precision    recall  f1-score   support

         0.0       0.67      0.67      0.67        36
         1.0       0.76      0.76      0.76        51

   micro avg       0.72      0.72      0.72        87
   macro avg       0.72      0.72      0.72        87
weighted avg       0.72      0.72      0.72        87

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


# Ensemble Tw, News and FI -Scaled

In [12]:
import pandas as pd

In [13]:
trainingData = pd.read_csv('train_tw_nw_fi.csv')
testingData =  pd.read_csv('test_tw_nw_fi.csv')

In [14]:
getStartEndDateShape(trainingData)
getStartEndDateShape(testingData)

startDate 2015-05-04
endDate 2018-12-31
shape (1336, 79)
---------------------------
startDate 2019-01-01
endDate 2019-03-28
shape (87, 79)
---------------------------


In [10]:
train, train_y = getFeaturesAndTargetRemoveDate(trainingData)
test, test_y = getFeaturesAndTargetRemoveDate(testingData)

In [21]:
NaiveBayes(train , train_y, test,test_y)

accuracy_score 0.4367816091954023
classification_score
               precision    recall  f1-score   support

         0.0       0.92      0.42      0.57        79
         1.0       0.10      0.62      0.17         8

   micro avg       0.44      0.44      0.44        87
   macro avg       0.51      0.52      0.37        87
weighted avg       0.84      0.44      0.54        87



In [130]:
train.head(2)

,Open,High,Low,Close,Adj Close,Volume,sma2,sma2_1,sma2_increment,sma2_1_increment,...,disgust_y,anticipation_y,fear_y,trust_y,tb_polarity_y,tb_subjectivity_y,hiv4_positive_y,hiv4_negative_y,hiv4_polarity_y,hiv4_subjectivity_y
0,-1.114678,-1.109405,-1.107374,-1.121569,-1.153836,0.233553,-1.109740,-1.100390,-0.308662,-0.196784,...,-0.965674,1.521927,-0.575468,-0.994978,-1.060730,-1.197176,-1.319448,-1.28863,-1.147992,-1.255845
1,-1.139229,-1.140805,-1.146727,-1.150169,-1.178850,1.278356,-1.126732,-1.107775,-0.592548,-0.328013,...,-0.965674,2.458020,-0.575468,-0.994978,-1.092967,-1.211317,-1.323400,-1.28863,-1.161587,-1.260280


In [131]:
z = cCMatrixAccXGB(train , train_y, test,test_y)

C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\praveen ji\Anaconda3\envs\tf_gpu\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model accuracy = 0.690
classification_score
               precision    recall  f1-score   support

         0.0       0.62      0.64      0.63        36
         1.0       0.74      0.73      0.73        51

   micro avg       0.69      0.69      0.69        87
   macro avg       0.68      0.68      0.68        87
weighted avg       0.69      0.69      0.69        87



In [132]:
features = list(train.columns)
k=cCMatrixAccRF(train , train_y, test,test_y, features,False)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
confusion_matrix [[24 12]
 [12 39]]
number of datapoints in Up Class 51
number of datapoints in Down Class 36
accuracy_score 0.7241379310344828
classification_score
               precision    recall  f1-score   support

         0.0       0.67      0.67      0.67        36
         1.0       0.76      0.76      0.76        51

   micro avg       0.72      0.72      0.72        87
   macro avg       0.72      0.72      0.72        87
weighted avg       0.72      0.72      0.72        87

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [133]:
## Future Work

In [134]:
# train_fiDf = train[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sma2', 'sma2_1',
#        'sma2_increment', 'sma2_1_increment', 'vol_increment',
#        'vol_rel_increment',  'open_incr', 'ma7', 'ma21',
#        '26ema', '12ema', 'MACD', '30 Day MA', '30 Day STD', 'upper_band',
#        'lower_band', 'ema', 'momentum', 'log_momentum', 'fft', 'absolute',
#        'angle']]
# test_fiDf = test[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sma2', 'sma2_1',
#        'sma2_increment', 'sma2_1_increment', 'vol_increment',
#        'vol_rel_increment',  'open_incr', 'ma7', 'ma21',
#        '26ema', '12ema', 'MACD', '30 Day MA', '30 Day STD', 'upper_band',
#        'lower_band', 'ema', 'momentum', 'log_momentum', 'fft', 'absolute',
#        'angle']]

In [135]:
# train_nwDf= train[[ 'lm_positive_x', 'lm_negative_x', 'lm_polarity_x',
#        'lm_subjectivity_x', 'actvie_x', 'passive_x', 'weak_x', 'strong_x',
#        'anger_x', 'joy_x', 'suprise_x', 'sadness_x', 'disgust_x',
#        'anticipation_x', 'fear_x', 'trust_x', 'tb_polarity_x',
#        'tb_subjectivity_x', 'hiv4_positive_x', 'hiv4_negative_x',
#        'hiv4_polarity_x', 'hiv4_subjectivity_x', 'lm_positive_y']]
# test_nwDf= test[[ 'lm_positive_x', 'lm_negative_x', 'lm_polarity_x',
#        'lm_subjectivity_x', 'actvie_x', 'passive_x', 'weak_x', 'strong_x',
#        'anger_x', 'joy_x', 'suprise_x', 'sadness_x', 'disgust_x',
#        'anticipation_x', 'fear_x', 'trust_x', 'tb_polarity_x',
#        'tb_subjectivity_x', 'hiv4_positive_x', 'hiv4_negative_x',
#        'hiv4_polarity_x', 'hiv4_subjectivity_x', 'lm_positive_y']]

In [136]:
# train_twDf= train[[ 'lm_negative_y', 'lm_polarity_y',
#        'lm_subjectivity_y', 'actvie_y', 'passive_y', 'weak_y', 'strong_y',
#        'anger_y', 'joy_y', 'suprise_y', 'sadness_y', 'disgust_y',
#        'anticipation_y', 'fear_y', 'trust_y', 'tb_polarity_y',
#        'tb_subjectivity_y', 'hiv4_positive_y', 'hiv4_negative_y', 
#        'retweet_count', 'hiv4_polarity_y', 'hiv4_subjectivity_y']]
# test_twDf= test[[ 'lm_negative_y', 'lm_polarity_y',
#        'lm_subjectivity_y', 'actvie_y', 'passive_y', 'weak_y', 'strong_y',
#        'anger_y', 'joy_y', 'suprise_y', 'sadness_y', 'disgust_y',
#        'anticipation_y', 'fear_y', 'trust_y', 'tb_polarity_y',
#        'tb_subjectivity_y', 'hiv4_positive_y', 'hiv4_negative_y', 
#        'retweet_count', 'hiv4_polarity_y', 'hiv4_subjectivity_y']]

In [137]:
# col_names =  ['news', 'tweeter', 'financialIndicator','target']
# YHat_test  = pd.DataFrame(columns = col_names)
# YHat_train = pd.DataFrame(columns = col_names)
# YHat_test['target'] = test_y
# YHat_train['target'] = train_y

In [138]:
# col_names =  ['news', 'tweeter', 'financialIndicator','target']
# YHat_rf_test  = pd.DataFrame(columns = col_names)
# YHat_rf_train = pd.DataFrame(columns = col_names)
# YHat_rf_test['target'] = test_y
# YHat_rf_train['target'] = train_y

In [139]:
# YHat_test['news'] = cCMatrixAccXGB(train_nwDf , train_y, test_nwDf,test_y)
# YHat_train['news'] = cCMatrixAccXGB(train_nwDf , train_y, train_nwDf,train_y)

In [140]:
# YHat_rf_train['news'], YHat_rf_test['news'] = cCMatrixAccRF(train_nwDf , train_y, test_nwDf,test_y, list(train.columns))

In [141]:
# YHat_test['tweeter'] = cCMatrixAccXGB(train_twDf , train_y, test_twDf,test_y)
# YHat_train['tweeter'] = cCMatrixAccXGB(train_twDf , train_y, train_twDf,train_y)

In [142]:
# YHat_test['tweeter'] = cCMatrixAccXGB(train_twDf , train_y, test_twDf,test_y)
# YHat_train['tweeter'] = cCMatrixAccXGB(train_twDf , train_y, train_twDf,train_y)

In [143]:
# YHat_test['financialIndicator'] = cCMatrixAccXGB(train_fiDf , train_y, test_fiDf,test_y)
# YHat_train['financialIndicator'] = cCMatrixAccXGB(train_fiDf , train_y, train_fiDf,train_y)

In [144]:
# YHat_test['financialIndicator'] = cCMatrixAccXGB(train_fiDf , train_y, test_fiDf,test_y)
# YHat_train['financialIndicator'] = cCMatrixAccXGB(train_fiDf , train_y, train_fiDf,train_y)

In [145]:
# YHat_train.shape

In [146]:
# YHat_test.shape

In [147]:
# temp = cCMatrixAccXGB(YHat_train.drop('target',axis =1) , YHat_train['target'],YHat_test.drop('target',axis =1) , YHat_test['target'])